# Importing packages

In [ ]:
# Required module imports
import sys
import csv
from time import sleep, strftime
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Creating  list of dates 

In [ ]:
import datetime
dates = []
start = datetime.datetime.strptime("07-01-2022", "%d-%m-%Y")
end = datetime.datetime.strptime("01-07-2022", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    dates.append(str(date.strftime("%Y-%m-%d")))

In [ ]:
date_data = []
for i,j in zip(date_generated,dates):
    date_data.append([i,j])

# Choose city

In [ ]:
# User defined variables for data retreival
city_from = "Hyderabad"             # Origin airport code
city_to = "Jaipur" # Destination airport code

# code to scrap data

In [5]:
for t in range(len(date_data)):
    
    # url to display data
    baseDataUrl =('https://www.kayak.co.in/flights/' + city_from + '-' + city_to +'/' + date_data[t][1] + '?sort=price_a')


    # webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome(executable_path="chromedriver.exe") # Chrome driver is being used.

    print ("Requesting URL: " + baseDataUrl)

    driver.get(baseDataUrl)   # URL requested in browser.
    print ("Webpage found ...")
    sleep(20)

    try:
        xp_popup_close = '//button[contains(@id,"-dialog-close") and contains(@class,"Button-No-Standard-Style close inside darkIcon ")]'
        l = len(driver.find_elements_by_xpath(xp_popup_close))
        driver.find_elements_by_xpath(xp_popup_close)[l-1].click()
    except Exception as e:
        pass

    element_xpath = '//div[contains(@class,"Base-Results-ResultsPage Flights-Results-FlightResultsPage ResultsVisible PhoenixRising narrow-padding")]' # First box with relevant flight data.

    # Wait until the first box with relevant flight data appears on Screen
    element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))
    sleep(10)

    try:
        print('show all results.....')
        show_all = '//a[@class="showAll"]'
        driver.find_element_by_xpath(show_all).click()
        print('done.....')
        sleep(15)
    except:
        pass

    #   click on show more 
    try:
        print('load more results.....')
        load_more = '//div[@class="resultsPaginator"]'
        driver.find_element_by_xpath(show_all).click()
        print('done.....')
        sleep(10)
    except:
        pass

    # Find the document body and get its inner HTML for processing in BeautifulSoup parser.
    body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

    print("Getting data from DOM ...")
    soupBody = BeautifulSoup(body,'html') # Parse the inner HTML using BeautifulSoup

    airline = soupBody.find_all("span", "codeshares-airline-names") 
    date_list = date_data[t][0].day
    weekday = date_data[t][0].weekday()
    month = date_data[t][0].month
    year = date_data[t][0].year
    depr_time = soupBody.find_all("span", "depart-time base-time")  
    ar_time = soupBody.find_all("span", "arrival-time base-time") 
    depr_city_name = city_from   

    ar_city_name = city_to 
    travel_time = soupBody.find_all("div", "section duration allow-multi-modal-icons")    
    airline_cabin = 'E'
    no_of_stops =soupBody.find_all("span", "stops-text")   
    prices_list = soupBody.find_all("span", "price option-text")  
    
    # Fixing price
    price = []
    for i in range(len(prices_list)):
        p = prices_list[i].text
        p = p.strip().replace(',','')[2:]
        price.append(p)

    flightsData = []
    flight_details = ["airlines" ,"Date", "Month", "Year","Weekday", "Dep_Time","Ar_Time", "depr_Citie", "Ar_Citie", "Travel_Time", "Airline_Cabin", "no_stops",  "Price"]

    # Extracting data from tags and appending to main database flightsData
    for i in range(0, len(airline)):
        flightsData.append([airline[i].text, date_list, month, year, weekday, depr_time[i].text, ar_time[i].text , depr_city_name , ar_city_name, travel_time[i].find('div','top').text.strip() , airline_cabin, no_of_stops[i].text.strip(), price[i] ] )

    flight_Df = pd.DataFrame(flightsData,columns=flight_details)

    flight_Df['timestamp'] = strftime("%Y%m%d-%H%M") # so we can know when it was scraped

    outputFile = "FlightsData_" + city_from +"-"+ city_to +"-"+ date_data[t][1] + ".csv"
    
    print("Writing flight data to file: "+ outputFile + " ...")
    flight_Df.to_csv("flight_data\\"+outputFile,index=False)
    
    print("Closing Chrome ...") # No more usage needed.
    driver.quit() # Browser Closed.


Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-01-07?sort=price_a
Webpage found ...
show all results.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-01-07.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-01-08?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-01-08.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-01-09?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-01-09.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-01-10?sort=price_a
Webpage found ...
show all results.....
done.....
load more results..

Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-02-05?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-02-05.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-02-06?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-02-06.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-02-07?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-02-07.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-02-08?sort=price_a
Webpage found ...
show all results.....
done.....
load more

Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-03-06?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-03-06.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-03-07?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-03-07.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-03-08?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-03-08.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-03-09?sort=price_a
Webpage found ...
show all results.....
done.....
load more

Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-04-04?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-04-04.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-04-05?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-04-05.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-04-06?sort=price_a
Webpage found ...
show all results.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-04-06.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-04-07?sort=price_a
Webpage found ...
show all results.....
done.....
load more results..

Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-05-03?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-05-03.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-05-04?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-05-04.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-05-05?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-05-05.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-05-06?sort=price_a
Webpage found ...
show all results.....
done.....
load more

Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-06-01?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-06-01.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-06-02?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-06-02.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-06-03?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-06-03.csv ...
Closing Chrome ...
Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-06-04?sort=price_a
Webpage found ...
show all results.....
done.....
load more

Requesting URL: https://www.kayak.co.in/flights/Hyderabad-Jaipur/2022-06-30?sort=price_a
Webpage found ...
show all results.....
done.....
load more results.....
Getting data from DOM ...
Writing flight data to file: FlightsData_Hyderabad-Jaipur-2022-06-30.csv ...
Closing Chrome ...
